## max_score + Hashing trick for look-alike portal application completed


In [1]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
#import numpy as np
#import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn
try:
    sc.stop()
except NameError: pass


conf = SparkConf().set("spark.executor.instances", 8).set("spark.driver.maxResultSize", "4g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

import pandas as pd
import numpy as np
import os
import sklearn
import re
import xgboost as xgb

/home/k.p.osminin/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = []
for r in open('/home/k.p.osminin/external_hdfs/ha_raw_lah_20161012.txt','r'):
    e = r.strip('\n').split(' ')[:3]
    train.append([int(e[0]),int(e[1]), float(e[2])])
        
print(len(train))
train_hash_pred = [float(e) for e in open('/home/k.p.osminin/external_hdfs/pred_20161012.txt','r')]
print(len(train_hash_pred))

7468151
7468151


In [3]:
df_train = pd.DataFrame(data = [e[0]+[e[1]] for e in zip(train,train_hash_pred)],columns = ['label','first_day','tcs_maxs','hash_score'])

In [4]:
dtrain = xgb.DMatrix( df_train[['tcs_maxs','hash_score']], label=df_train.label)

In [5]:
param = {'max_depth':10, 'eta':1, 'gamma' : 0.5, 'silent':1, 'objective':'binary:logistic','eval_metric' : "auc",
         'scale_pos_weight' : 1, 'num_round' : 10 }

bst = xgb.train(param, dtrain)

# make prediction
df_train['xgboost_pred'] = bst.predict(dtrain)

In [9]:

print('{0} AUCROC train: tcs_max_score {1:.6f}. ha_score {2:.6f}. xgboost(tcs_max,ha_sc) {3:.6f}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = df_train.tcs_maxs
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = df_train.hash_score
                ),                                              
               sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = df_train.xgboost_pred
                ),                                              
))

XGBoost AUCROC train: tcs_max_score 0.855961. ha_score 0.837702. xgboost(tcs_max,ha_sc) 0.877686


In [10]:
test = []
for r in open('/home/k.p.osminin/external_hdfs/ha_raw_lah_20161019.txt','r'):
    e = r.strip('\n').split(' ')[:3]
    test.append([int(e[0]),int(e[1]), float(e[2])])
        
print(len(test))
test_hash_pred = [float(e) for e in open('/home/k.p.osminin/external_hdfs/pred_20161019.txt','r')]
print(len(test_hash_pred))

7476264
7476264


In [11]:
df_test = pd.DataFrame(data = [e[0]+[e[1]] for e in zip(test,test_hash_pred)],columns = ['label','first_day','tcs_maxs','hash_score'])
df_test['xgboost_pred'] = bst.predict(xgb.DMatrix( df_test[['tcs_maxs','hash_score']], label=df_test.label))

In [12]:
print('{0} AUCROC test: tcs_max_score {1:.6f}. ha_score {2:.6f}. xgboost(tcs_max,ha_sc) {3:.6f}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = df_test.tcs_maxs
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = df_test.hash_score
                ),                                              
               sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = df_test.xgboost_pred
                ),                                              
))

XGBoost AUCROC test: tcs_max_score 0.852580. ha_score 0.827884. xgboost(tcs_max,ha_sc) 0.850725


In [18]:
df_train.loc[:,'tcs_maxs_bin'] = pd.cut(
    df_train['tcs_maxs'], 
    bins = np.unique(df_train['tcs_maxs'].quantile(np.arange(0,1.05,0.05)).values), 
    labels  = False).values

df_train.loc[:,'hash_score_bin'] = pd.cut(
    df_train['hash_score'], 
    bins = np.unique(df_train['hash_score'].quantile(np.arange(0,1.05,0.05)).values), 
    labels  = False).values

In [161]:
#tab = df_train.groupby(['tcs_maxs_bin','hash_score_bin'])['label'].agg([np.mean,np.size]).unstack() 
tab['size']

hash_score_bin,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
tcs_maxs_bin,,,,,,,,,,,,,,
0.0,NaN,220.0,53.0,81400.0,18786.0,272571.0,122.0,195.0,324.0,24.0,19.0,4.0,5.0,4.0
1.0,226599.0,222511.0,153802.0,13598.0,16543.0,91069.0,94.0,113.0,119.0,15.0,19.0,7.0,2.0,4.0
2.0,4.0,263.0,19.0,399.0,2760.0,18864.0,30.0,23.0,29.0,3.0,2.0,1.0,3.0,NaN
3.0,66.0,6528.0,4452.0,10568.0,28820.0,317948.0,544.0,3390.0,411.0,80.0,69.0,28.0,52.0,18.0
4.0,151569.0,129892.0,2051.0,5377.0,44952.0,169633.0,127831.0,1155.0,499.0,121.0,66.0,44.0,62.0,16.0
5.0,5015.0,2637.0,271746.0,119089.0,125960.0,105225.0,2793.0,305.0,160.0,55.0,117.0,63.0,108.0,8.0
6.0,286.0,2272.0,5632.0,4861.0,10654.0,179559.0,11670.0,10123.0,782.0,1079.0,71.0,39.0,44.0,20.0
7.0,112.0,2070.0,4416.0,4510.0,17853.0,231335.0,18299.0,46657.0,32451.0,16225.0,5185.0,1855.0,813.0,423.0
8.0,151.0,2611.0,3780.0,4324.0,10749.0,260123.0,17839.0,23478.0,37244.0,9561.0,6206.0,3150.0,1402.0,441.0


In [ ]:

print('{0} AUCROC train: tcs_max_score {1:.6f}. ha_score {2:.6f}. xgboost(tcs_max,ha_sc) {3:.6f}'.format(
               'RandomForest',
                sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = [e[1] for e in clfRF.predict_proba(df_train[['tcs_maxs','hash_score']])]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = [e[1] for e in clfRF.predict_proba(df_test[['tcs_maxs','hash_score']])]
                ),
))


In [47]:
df_train_top = df_train[df_train['tcs_maxs'] > df_train['tcs_maxs'].quantile(0.95)]

In [48]:
df_train_top.loc[:,'hash_score_bin'] = pd.cut(
    df_train_top['hash_score'], 
    bins = np.unique(df_train_top['hash_score'].quantile(np.arange(0,1.01,0.1)).values), 
    labels  = False).values

In [49]:
df_train_top.groupby('hash_score_bin')['label'].agg([np.mean,np.size])

,mean,size
hash_score_bin,,
0.0,0.001578,46896
1.0,0.002484,27779
2.0,0.002057,37428
3.0,0.002604,37244
4.0,0.003053,37337
5.0,0.003294,37339
6.0,0.003884,37335
7.0,0.004794,37337
8.0,0.006187,37337


In [64]:
df_train_sampled = df_train[(df_train.label == 1) | (np.random.rand(len(df_train)) < 0.1)]

In [68]:
import sklearn.ensemble
from sklearn import svm
clf = {
   # 'LinSVC': sklearn.svm.LinearSVC(penalty='l1', loss='squared_hinge',C=1.0, class_weight='auto', max_iter=1000),
   #'SVC': sklearn.svm.SVC(probability = True,max_iter = 100),
    'LogRegr': sklearn.linear_model.LogisticRegression(),
    'RandomForest': sklearn.ensemble.RandomForestClassifier(max_depth = 5,n_estimators = 50),
   # 'GBM': sklearn.ensemble.GradientBoostingClassifier(n_estimators = 400)
    
}
for m in clf:
    clf[m].fit(X = df_train[['tcs_maxs','hash_score']],y = df_train['label'])
    print('{0} AUCROC: {1}'.format(m,sklearn.metrics.roc_auc_score(
                y_true = df_test['label'], 
                y_score = [e[1] for e in clf[m].predict_proba(df_test[['tcs_maxs','hash_score']])]
    )))

RandomForest AUCROC: 0.858583009817
LogRegr AUCROC: 0.853759819568



#### RandomForest Немного лучше max_tcs_score на тесте.

In [110]:
#a = np_utils.normalize(df_train[['tcs_maxs','hash_score']])
from sklearn import preprocessing
import numpy as np
X = np.array([[ 1., -1.,  2.,11],
               [ 2.,  0.,  0.,13],
               [ 0.,  1., -1.,10]])
scaler = preprocessing.StandardScaler().fit(df_train_sampled[['tcs_maxs','hash_score']])
X, y = scaler.transform(df_train_sampled[['tcs_maxs','hash_score']]), df_train_sampled['label']
Xt, yt = scaler.transform(df_test[['tcs_maxs','hash_score']]), df_test['label']


Y_train = np_utils.to_categorical(df_train_sampled['label'].values, 2) # One-hot encode the labels
#Y_test = np_utils.to_categorical(df_test['label'], 2) # One-hot encode the labels

In [216]:
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense # the two types of neural network layer we will be using
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values


batch_size = 256
num_epochs = 15 # we iterate twenty times over the entire training set
hidden_size_1 = 32 # there will be 16 neurons in first layer
hidden_size_2 = 16 


inp = Input(shape=(2,)) # Our input is a 1D vector of size 784
hidden_1 = Dense(hidden_size_1, activation='relu')(inp) # First hidden ReLU layer
hidden_2 = Dense(hidden_size_2, activation='relu')(hidden_1) # Second hidden ReLU layer
out = Dense(2, activation='softmax')(hidden_2) # Output softmax layer

model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

In [217]:
model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [218]:
model.fit(X, Y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
#model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Train on 675550 samples, validate on 75062 samples
Epoch 1/15
675550/675550 [==============================] - 6s - loss: 0.0438 - acc: 0.9919 - val_loss: 0.0288 - val_acc: 0.9942
Epoch 2/15
675550/675550 [==============================] - 6s - loss: 0.0277 - acc: 0.9944 - val_loss: 0.0287 - val_acc: 0.9942
Epoch 3/15
675550/675550 [==============================] - 6s - loss: 0.0277 - acc: 0.9944 - val_loss: 0.0291 - val_acc: 0.9939
Epoch 4/15
675550/675550 [==============================] - 6s - loss: 0.0277 - acc: 0.9944 - val_loss: 0.0288 - val_acc: 0.9942
Epoch 5/15
675550/675550 [==============================] - 7s - loss: 0.0277 - acc: 0.9945 - val_loss: 0.0291 - val_acc: 0.9940
Epoch 6/15
675550/675550 [==============================] - 8s - loss: 0.0276 - acc: 0.9945 - val_loss: 0.0289 - val_acc: 0.9943
Epoch 7/15
675550/675550 [==============================] - 6s - loss: 0.0276 - acc: 0.9945 - val_loss: 0.0288 - val_acc: 0.9942
Epoch 8/15
675550/675550 [====================

In [219]:
    print('{0} AUCROC: {1}'.format('Keras train',sklearn.metrics.roc_auc_score(
                y_true = y,
                y_score =[e[1] for e in model.predict(X)]
    )))


Keras train AUCROC: 0.868020784375


In [220]:
    print('{0} AUCROC: {1}'.format('Keras test',sklearn.metrics.roc_auc_score(
                y_true = yt,
                y_score = [e[1] for e in model.predict(Xt)]
    )))

#model.predict(Xt)

Keras test AUCROC: 0.86265400094


In [138]:
from sklearn.ensemble import AdaBoostClassifier

cl = AdaBoostClassifier()
cl.fit(df_train[['tcs_maxs','hash_score']], df_train['label'])

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [140]:
print('{0} AUCROC train: {1:.6f}. test: {2:.6f}'.format(
               'AdaBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = df_train.label, 
                  y_score = cl.predict_proba(df_train[['tcs_maxs','hash_score']])[:,1]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_test.label, 
                  y_score = cl.predict_proba(df_test[['tcs_maxs','hash_score']])[:,1]
                ),
))


AdaBoost AUCROC train: 0.869603. test: 0.861106


In [156]:
from sklearn.ensemble import AdaBoostClassifier
print('Target class is first day.')
clAB1 = AdaBoostClassifier()
clAB1.fit(df_train[['tcs_maxs','hash_score']], df_train['first_day'])
print('{0} AUCROC train: {1:.6f}. test: {2:.6f}'.format(
               'AdaBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = df_train['first_day'], 
                  y_score = clAB1.predict_proba(df_train[['tcs_maxs','hash_score']])[:,1]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = df_test['first_day'], 
                  y_score = clAB1.predict_proba(df_test[['tcs_maxs','hash_score']])[:,1]
                ),
)) 

AdaBoost AUCROC train: 0.877821. test: 0.868445


In [184]:
print('{:<30}  {:<10}  {:<10}  {:<10}'.format('Sample','label sum','count','label mean'))
print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('Full train sample', df_train.label.sum(),df_train.label.count(),df_train.label.mean()))
for q in [0.7,0.8,0.9,0.95,0.98,0.99]:
    df1 = df_train[(df_train['tcs_maxs'] > df_train['tcs_maxs'].quantile(q)) | (df_train['hash_score'] > df_train['hash_score'].quantile(q))]
    print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('tcs_maxs >q{0} or ha >q{0}'.format(q), df1.label.sum(),df1.label.count(),df1.label.mean()))
    df1 = df_train[(df_train['tcs_maxs'] > df_train['tcs_maxs'].quantile(q)) & (df_train['hash_score'] > df_train['hash_score'].quantile(q))]
    print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('tcs_maxs >q{0} and ha >q{0}'.format(q), df1.label.sum(),df1.label.count(),df1.label.mean()))


#df_train_top1.label.sum(),df_train_top1.label.mean()

Sample                          label sum   count       label mean
Full train sample               4340        7468151     0.0006    
tcs_maxs >q0.7 or ha >q0.7      3879        2825140     0.0014    
tcs_maxs >q0.7 and ha >q0.7     3187        1552765     0.0021    
tcs_maxs >q0.8 or ha >q0.8      3530        2029848     0.0017    
tcs_maxs >q0.8 and ha >q0.8     2630        957364      0.0027    
tcs_maxs >q0.9 or ha >q0.9      2979        1123401     0.0027    
tcs_maxs >q0.9 and ha >q0.9     1846        364596      0.0051    
tcs_maxs >q0.95 or ha >q0.95    2338        601552      0.0039    
tcs_maxs >q0.95 and ha >q0.95   1279        145225      0.0088    
tcs_maxs >q0.98 or ha >q0.98    1673        254736      0.0066    
tcs_maxs >q0.98 and ha >q0.98   818         43970       0.0186    
tcs_maxs >q0.99 or ha >q0.99    1274        134312      0.0095    
tcs_maxs >q0.99 and ha >q0.99   601         14978       0.0401    


In [187]:
print('{:<30}  {:<10}  {:<10}  {:<10}'.format('Sample','label sum','count','label mean'))
print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('Full train sample', df_train.label.sum(),df_train.label.count(),df_train.label.mean()))
for q in [0.7,0.8,0.9,0.95,0.98,0.99,0.995,0.999]:
    df1 = df_train[(df_train['tcs_maxs'] > df_train['tcs_maxs'].quantile(q))]
    print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('tcs_maxs >q{0}'.format(q), df1.label.sum(),df1.label.count(),df1.label.mean()))


Sample                          label sum   count       label mean
Full train sample               4340        7468151     0.0006    
tcs_maxs >q0.7                  3555        2138254     0.0017    
tcs_maxs >q0.8                  3117        1493582     0.0021    
tcs_maxs >q0.9                  2418        741182      0.0033    
tcs_maxs >q0.95                 1853        373370      0.0050    
tcs_maxs >q0.98                 1303        149343      0.0087    
tcs_maxs >q0.99                 986         74608       0.0132    
tcs_maxs >q0.995                826         36200       0.0228    
tcs_maxs >q0.999                652         7477        0.0872    


In [191]:
print('{:<30}  {:<10}  {:<10}  {:<10}'.format('Sample','label sum','count','label mean'))
print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('Full train sample', df_train.label.sum(),df_train.label.count(),df_train.label.mean()))
for q in [0.7,0.8,0.9,0.95,0.98,0.99,0.995,0.999]:
    df1 = df_train[df_train['hash_score'] > df_train['hash_score'].quantile(q)]
    print('{:<30}  {:<10}  {:<10}  {:<10.4f}'.format('ha >q{0}'.format(q), df1.label.sum(),df1.label.count(),df1.label.mean()))


Sample                          label sum   count       label mean
Full train sample               4340        7468151     0.0006    
ha >q0.7                        3511        2239651     0.0016    
ha >q0.8                        3043        1493630     0.0020    
ha >q0.9                        2407        746815      0.0032    
ha >q0.95                       1764        373407      0.0047    
ha >q0.98                       1188        149363      0.0080    
ha >q0.99                       889         74682       0.0119    
ha >q0.995                      694         37341       0.0186    
ha >q0.999                      417         7469        0.0558    
